In [1]:
%run BandPassFilter.ipynb

def RPeakDetector(signal, frac, win_l_calib, win_l_peak_search, fs):        
    k_BP = .7     # cut-off value
    flc = 15/fs
    ecg_bp1 = BandPassFilter(signal, k_BP, flc)    
    #
    k_BP = .7     # cut-off value    
    fuc = 45/fs
    ecg_bp = BandPassFilter(ecg_bp1, k_BP, fuc)
    #
    k_tan = 4
    alpha = k_tan * np.std(ecg_bp)
    ecg_tan = alpha * np.tanh(ecg_bp/alpha)
    #
    #ecg_sq = ecg_tan**2
    ecg_sq = np.abs(ecg_tan)
    #
    wind_len_MA = .1
    ecg_MA_sq = ecg_sq
    #ecg_MA_sq = np.sqrt(func_MA_filter(ecg_sq, wind_len_MA, fs))
    #
    maxecg = max(ecg_MA_sq)
    heightlim = frac * maxecg
    peaks, _ = scsig.find_peaks(ecg_MA_sq, distance = win_l_peak_search*fs, height = heightlim)            
    #
    #print("peaks raw (as per the detector)", peaks)
    wind_len_hf = int(win_l_calib * fs)
    pks = []
    for i in range(len(peaks)):
        if(i == 0):
            wind = np.arange(peaks[i], peaks[i]+wind_len_hf, 1)
            ecg_bwr_win = signal[wind]
            p = peaks[i] + np.argmax(ecg_bwr_win**2)   
        elif(i == len(peaks) - 1):
            wind = np.arange(peaks[i] - wind_len_hf, peaks[i], 1)            
            ecg_bwr_win = signal[wind]
            p = peaks[i]- wind_len_hf + np.argmax(ecg_bwr_win**2)                
        else:
            wind = np.arange(peaks[i] - wind_len_hf, peaks[i] + wind_len_hf, 1)
            ecg_bwr_win = signal[wind]
            p = peaks[i]- wind_len_hf + np.argmax(ecg_bwr_win**2)                            
        pks.append(p)    

    return(ecg_bp1, ecg_bp, ecg_tan, ecg_sq, ecg_MA_sq, peaks, np.array(pks))